## Deploying a Decision Optimization model with Watson Machine Learning

This notebook shows you how to deploy a Decision Optimization model written in Python, handle its inputs data, create and monitor jobs, and get solutions using the Watson Machine Learning Python Client.



## Table of Contents
1. [Install the Watson Machine Learning client API](#setup)
2. [Create a client instance](#create)
3. [Prepare your model archive](#prepare)
4. [Upload your model on Watson Machine Learning](#upload)
5. [Create a deployment](#deploy)
6. [Create and monitor a job with inline data for your deployed model](#job)
7. [Display the solution](#display)
8. [Solve another problem using the same deployment](#problem)
9. [Summary](#summary)

<a id='setup'></a>
### Set up the Watson Machine Learning client

Before you use the sample code in this notebook, you need to:

- create a <a href="https://cloud.ibm.com/catalog/services/machine-learning" target="_blank" rel="noopener noreferrer">Watson Machine Learning (WML) Service</a> instance. A free plan is offered and information about how to create the instance can be found at <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/wml-setup.html" target="_blank" rel="noopener noreferrer"> Create a WML instance</a>.



Install and then import the Watson Machine Learning client library. This notebook uses the preview Python client based on v4 of Watson Machine Learning APIs. 

**Important** Do not load both Python client libraries into a notebook.

In [1]:
# Uninstall the Watson Machine Learning client Python client based on v3 APIs

!pip uninstall watson-machine-learning-client -y

In [2]:
# Install the WML client API

!pip install watson-machine-learning-client-V4

In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

<a id='create'></a>
### Create a client instance

Use your Watson Machine Learning credentials.

In [4]:
# Instantiate a client using credentials
wml_credentials = {
      "apikey": "XXXXXXXXXXX",
      "instance_id": "XXXXXXXXXXXXX",
      "url": "https://us-south.ml.cloud.ibm.com",
}

In [6]:


client = WatsonMachineLearningAPIClient(wml_credentials)

In [7]:
client.version

'1.0.34'

<a id='prepare'></a>
### Prepare your model archive

Put the model.py file in a subdirectory and create a tar.gz file. The model consists of two parts:
* some functions to create an `inputs` dictionary from files and create files from an `outputs` dictionary,
* the real optimization model which uses the inputs and outputs dictionaries.

Use the `write_file` command to write these models to a `main.py` file. 

Use the `tar` command to create a tar archive.

In [8]:
%mkdir model

mkdir: cannot create directory ‘model’: File exists


In [9]:
%%writefile model/main.py

from docplex.util.environment import get_environment
from os.path import splitext
import pandas
from six import iteritems

def get_all_inputs():
    '''Utility method to read a list of files and return a tuple with all
    read data frames.
    Returns:
        a map { datasetname: data frame }
    '''
    result = {}
    env = get_environment()
    for iname in [f for f in os.listdir('.') if splitext(f)[1] == '.csv']:
        with env.get_input_stream(iname) as in_stream:
            df = pandas.read_csv(in_stream)
            datasetname, _ = splitext(iname)
            result[datasetname] = df
    return result

def write_all_outputs(outputs):
    '''Write all dataframes in ``outputs`` as .csv.

    Args:
        outputs: The map of outputs 'outputname' -> 'output df'
    '''
    for (name, df) in iteritems(outputs):
        csv_file = '%s.csv' % name
        print(csv_file)
        with get_environment().get_output_stream(csv_file) as fp:
            if sys.version_info[0] < 3:
                fp.write(df.to_csv(index=False, encoding='utf8'))
            else:
                fp.write(df.to_csv(index=False).encode(encoding='utf8'))
    if len(outputs) == 0:
        print("Warning: no outputs written")
        

Overwriting model/main.py


In [10]:
%%writefile -a model/main.py

# Load CSV files into inputs dictionnary
inputs = get_all_inputs()

food = inputs['diet_food']
nutrients = inputs['diet_nutrients']
food_nutrients = inputs['diet_food_nutrients']
food_nutrients.set_index('Food', inplace=True)
        
from docplex.mp.model import Model

# Model
mdl = Model(name='diet')

# Create decision variables, limited to be >= Food.qmin and <= Food.qmax
qty = food[['name', 'qmin', 'qmax']].copy()
qty['var'] = qty.apply(lambda x: mdl.continuous_var(lb=x['qmin'],
                                                ub=x['qmax'],
                                                name=x['name']),
                   axis=1)
# make the name the index
qty.set_index('name', inplace=True)

# Limit range of nutrients, and mark them as KPIs
for n in nutrients.itertuples():
    amount = mdl.sum(qty.loc[f.name]['var'] * food_nutrients.loc[f.name][n.name]
                     for f in food.itertuples())
    mdl.add_range(n.qmin, amount, n.qmax)
    mdl.add_kpi(amount, publish_name='Total %s' % n.name)

# Minimize cost
obj = mdl.sum(qty.loc[f.name]['var'] * f.unit_cost for f in food.itertuples())
mdl.add_kpi(obj, publish_name="Minimal cost");
mdl.minimize(obj)

mdl.print_information()

# solve
ok = mdl.solve()

mdl.print_solution()

import pandas
import numpy

solution_df = pandas.DataFrame(columns=['Food', 'value'])

for index, dvar in enumerate(mdl.solution.iter_variables()):
    solution_df.loc[index,'Food'] = dvar.to_string()
    solution_df.loc[index,'value'] = dvar.solution_value
    
outputs = {}
outputs['solution'] = solution_df
        
# Generate output files
write_all_outputs(outputs)

Appending to model/main.py


> Looking at local files

In [11]:
!pwd ; echo ' -- '; find . -print

/home/dsxuser/work
 -- 
.
./model.tar.gz
./model
./model/main.py
./diet_food.csv
./diet_food_nutrients.csv
./diet_nutrients.csv


In [12]:
import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model/main.py", arcname="main.py", filter=reset)
tar.close()

<a id='upload'></a>
### Upload your model on Watson Machine Learning

Store model in Watson Machine Learning with:
* the tar archive previously created,
* metadata including the model type and runtime

Get the `model_uid`.

Documentation of the API is refered at : https://dataplatform.cloud.ibm.com/docs/content/DO/WML_Deployment/DeployPythonClient.html

In [13]:
# All available meta data properties 

client.repository.ModelMetaNames.show()

------------------------  ----  --------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
META_PROP NAME            TYPE  REQUIRED  SCHEMA
NAME                      str   Y
DESCRIPTION               str   N
INPUT_DATA_SCHEMA         dict  N         {'id(required)': 'string', 'fields(required)': [{'name(required)': 'string', 'type(required)': 'string', 'nullable(optional)': 'string'}]}
TRAINING_DATA_REFERENCES  list  N         [{'name(optional)': 'string', 'type(required)': 'string', 'connection(required)': {'endpoint_url(required)': 'string', 'access_key_id(required)': 'string', 'secret_access_key(required)': 'string'},

In [14]:
# All available runtimes

client.runtimes.list(pre_defined=True)

--------------------------  --------------------------  ------------------------  --------
GUID                        NAME                        CREATED                   PLATFORM
xgboost_0.82-py3.6          xgboost_0.82-py3.6          2019-08-01T07:55:19.417Z  python
xgboost_0.80-py3.6          xgboost_0.80-py3.6          2019-08-01T07:55:15.061Z  python
scikit-learn_0.20-py3.6     scikit-learn_0.20-py3.6     2019-08-01T07:55:04.628Z  python
scikit-learn_0.19-py3.6     scikit-learn_0.19-py3.6     2019-08-01T07:55:00.294Z  python
tensorflow_1.13-py3.6       tensorflow_1.13-py3.6       2019-08-01T07:54:49.668Z  python
tensorflow_1.11-py3.6       tensorflow_1.11-py3.6       2019-08-01T07:54:46.658Z  python
tensorflow_1.5-py3.6        tensorflow_1.5-py3.6        2019-08-01T07:54:19.401Z  python
ai-function_0.1-py3.6       ai-function_0.1-py3.6       2019-07-05T08:54:54.841Z  python
spss-modeler_18.2           spss-modeler_18.2           2019-07-05T08:54:49.935Z  spss
xgboost_0.82-py3   

In [15]:
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "Diet",
    client.repository.ModelMetaNames.DESCRIPTION: "Model for Diet Problem",
    client.repository.ModelMetaNames.TYPE: "do-docplex_12.9",
    client.repository.ModelMetaNames.RUNTIME_UID: "do_12.9"    
}

model_details = client.repository.store_model(model='/home/dsxuser/work/model.tar.gz', meta_props=mnist_metadata)

model_uid = client.repository.get_model_uid(model_details)

print( model_uid )

c6b80cb1-33f3-4400-a4da-3fbbf545caf6


<a id='deploy'></a>
### Create a deployment 

Create a batch deployment for the model, providing information such as:
* the maximum number of compute nodes
* the T-shirt size of the compute nodes

Get the `deployment_uid`.

In [16]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Diet Deployment",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "Diet Deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.COMPUTE: {'name': 'S', 'nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid = client.deployments.get_uid(deployment_details)

print( deployment_uid )



#######################################################################################

Synchronous deployment creation for uid: 'c6b80cb1-33f3-4400-a4da-3fbbf545caf6' started

#######################################################################################


ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='db97f64f-61a0-4e36-9277-603370ca31df'
------------------------------------------------------------------------------------------------


db97f64f-61a0-4e36-9277-603370ca31df


In [17]:
# List all existing deployments

client.deployments.list()

------------------------------------  ---------------  -----  ------------------------  -------------
GUID                                  NAME             STATE  CREATED                   ARTIFACT_TYPE
db97f64f-61a0-4e36-9277-603370ca31df  Diet Deployment  ready  2019-09-26T23:52:55.275Z  model
69c58007-1f83-4a03-a0c3-1e20a26e986a  Diet Deployment  ready  2019-09-26T23:07:39.902Z  model
3f5bfca3-1c17-41d6-8133-605d7aad8804  Diet Deployment  ready  2019-09-26T23:06:38.947Z  model
f5197564-60cf-41a8-ba3e-dffeeffd0a80  Diet Deployment  ready  2019-09-26T23:06:30.679Z  model
500ca59f-d050-4ede-9d39-42890fa8e1f5  Diet Deployment  ready  2019-09-26T21:08:45.983Z  model
0e55b23f-2204-4cbf-b4cf-1e3e9e3622d6  Diet Deployment  ready  2019-09-26T21:08:35.978Z  model
------------------------------------  ---------------  -----  ------------------------  -------------


<a id='job'></a>
### Create and monitor a job with inline data for your deployed model

Get input data as csv files or from any other data source such as **databases either on local or on cloud**.
Here csv files are retrieved via github.

Create a payload containing inline input data.

Create a new job with this payload and the deployment.

Get the `job_uid`.

In [18]:
!rm -rf ./data  ; git clone   https://github.com/michel-mle/data .

fatal: destination path '.' already exists and is not an empty directory.


In [19]:
!head -100  *.csv 

==> diet_food.csv <==
name,unit_cost,qmin,qmax
Roasted Chicken,0.84,0,10
Spaghetti W/ Sauce,0.78,0,10
"Tomato,Red,Ripe,Raw",0.27,0,10
"Apple,Raw,W/Skin",0.24,0,10
Grapes,0.32,0,10
Chocolate Chip Cookies,0.03,0,10
Lowfat Milk,0.23,0,10
Raisin Brn,0.34,0,10
Hotdog,0.31,0,10

==> diet_food_nutrients.csv <==
Food,Calories,Calcium,Iron,Vit_A,Dietary_Fiber,Carbohydrates,Protein
Spaghetti W/ Sauce,358.2,80.2,2.3,3055.2,11.6,58.3,8.2
Roasted Chicken,277.4,21.9,1.8,77.4,0.0,0.0,42.2
"Tomato,Red,Ripe,Raw",25.8,6.2,0.6,766.3,1.4,5.7,1.0
"Apple,Raw,W/Skin",81.4,9.7,0.2,73.1,3.7,21.0,0.3
Grapes,15.1,3.4,0.1,24.0,0.2,4.1,0.2
Chocolate Chip Cookies,78.1,6.2,0.4,101.8,0.0,9.3,0.9
Lowfat Milk,121.2,296.7,0.1,500.2,0.0,11.7,8.1
Raisin Brn,115.1,12.9,16.8,1250.2,4.0,27.9,4.0
Hotdog,242.1,23.5,2.3,0.0,0.0,18.0,10.4

==> diet_nutrients.csv <==
name,qmin,qmax
Calories,2000,2500
Calcium,800,1600
Iron,10,30
Vit_A,5000,50000
Dietary_Fiber,25,100
Carbohydrates,0,300
Protein,50,100


In [20]:
# Import pandas library 
import pandas as pd 

diet_food = pd.read_csv("diet_food.csv")
diet_nutrients = pd.read_csv("diet_nutrients.csv")
diet_food_nutrients = pd.read_csv("diet_food_nutrients.csv")

In [21]:
diet_food

,name,unit_cost,qmin,qmax
0,Roasted Chicken,0.84,0,10
1,Spaghetti W/ Sauce,0.78,0,10
2,"Tomato,Red,Ripe,Raw",0.27,0,10
3,"Apple,Raw,W/Skin",0.24,0,10
4,Grapes,0.32,0,10
5,Chocolate Chip Cookies,0.03,0,10
6,Lowfat Milk,0.23,0,10
7,Raisin Brn,0.34,0,10
8,Hotdog,0.31,0,10


In [22]:
diet_nutrients

,name,qmin,qmax
0,Calories,2000,2500
1,Calcium,800,1600
2,Iron,10,30
3,Vit_A,5000,50000
4,Dietary_Fiber,25,100
5,Carbohydrates,0,300
6,Protein,50,100


In [23]:
diet_food_nutrients

,Food,Calories,Calcium,Iron,Vit_A,Dietary_Fiber,Carbohydrates,Protein
0,Spaghetti W/ Sauce,358.2,80.2,2.3,3055.2,11.6,58.3,8.2
1,Roasted Chicken,277.4,21.9,1.8,77.4,0.0,0.0,42.2
2,"Tomato,Red,Ripe,Raw",25.8,6.2,0.6,766.3,1.4,5.7,1.0
3,"Apple,Raw,W/Skin",81.4,9.7,0.2,73.1,3.7,21.0,0.3
4,Grapes,15.1,3.4,0.1,24.0,0.2,4.1,0.2
5,Chocolate Chip Cookies,78.1,6.2,0.4,101.8,0.0,9.3,0.9
6,Lowfat Milk,121.2,296.7,0.1,500.2,0.0,11.7,8.1
7,Raisin Brn,115.1,12.9,16.8,1250.2,4.0,27.9,4.0
8,Hotdog,242.1,23.5,2.3,0.0,0.0,18.0,10.4


> Prepare the payload to pass input data to WML

In [24]:
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
        {
            "id":"diet_food.csv",
            "values" : diet_food
        },
        {
            "id":"diet_food_nutrients.csv",
            "values" : diet_food_nutrients
        },
        {
            "id":"diet_nutrients.csv",
            "values" : diet_nutrients
        }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
    {
        "id":".*\.csv"
    }
    ]
}

job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print( job_uid )

09dcf02f-544b-4a73-985a-882edf26f49b


In [25]:
deployment_uid

'db97f64f-61a0-4e36-9277-603370ca31df'

In [26]:
len(job_details['entity']['decision_optimization']['input_data'])
for i in range(3): 
    print(job_details['entity']['decision_optimization']['input_data'][i], "\n")

{'id': 'diet_food.csv', 'fields': ['name', 'unit_cost', 'qmin', 'qmax'], 'values': [['Roasted Chicken', 0.84, 0, 10], ['Spaghetti W/ Sauce', 0.78, 0, 10], ['Tomato,Red,Ripe,Raw', 0.27, 0, 10], ['Apple,Raw,W/Skin', 0.24, 0, 10], ['Grapes', 0.32, 0, 10], ['Chocolate Chip Cookies', 0.03, 0, 10], ['Lowfat Milk', 0.23, 0, 10], ['Raisin Brn', 0.34, 0, 10], ['Hotdog', 0.31, 0, 10]]} 

{'id': 'diet_food_nutrients.csv', 'fields': ['Food', 'Calories', 'Calcium', 'Iron', 'Vit_A', 'Dietary_Fiber', 'Carbohydrates', 'Protein'], 'values': [['Spaghetti W/ Sauce', 358.2, 80.2, 2.3, 3055.2, 11.6, 58.3, 8.2], ['Roasted Chicken', 277.4, 21.9, 1.8, 77.4, 0.0, 0.0, 42.2], ['Tomato,Red,Ripe,Raw', 25.8, 6.2, 0.6, 766.3, 1.4, 5.7, 1.0], ['Apple,Raw,W/Skin', 81.4, 9.7, 0.2, 73.1, 3.7, 21.0, 0.3], ['Grapes', 15.1, 3.4, 0.1, 24.0, 0.2, 4.1, 0.2], ['Chocolate Chip Cookies', 78.1, 6.2, 0.4, 101.8, 0.0, 9.3, 0.9], ['Lowfat Milk', 121.2, 296.7, 0.1, 500.2, 0.0, 11.7, 8.1], ['Raisin Brn', 115.1, 12.9, 16.8, 1250.2, 4.

Display job status until it is completed.

The first job of a new deployment might take some time as a compute node must be started.

In [27]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
queued...
queued...
queued...
queued...
running...
completed


<a id='display'></a>
### Extract and display solution

Display the output solution.

Display the KPI Total Calories value.

In [28]:
# Create a dataframe for the solution
solution = pd.DataFrame(job_details['entity']['decision_optimization']['output_data'][0]['values'], 
                        columns = job_details['entity']['decision_optimization']['output_data'][0]['fields'])
solution.head()

,Food,value
0,Spaghetti W/ Sauce,2.155172
1,Chocolate Chip Cookies,10.000000
2,Lowfat Milk,1.831167
3,Hotdog,0.929698


In [29]:
print( job_details['entity']['decision_optimization']['solve_state']['details']['KPI.Total Calories'] )

2000.0


<a id='problem'></a>
###  Solve another problem using the same deployment

Create a new payload with modified input data.

In [30]:
# Change the input data
diet_nutrients.at[0,'qmin'] = 1500
diet_nutrients.at[0,'qmax'] = 2000

solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
        {
            "id":"diet_food.csv",
            "values" : diet_food         
        },
        {
            "id":"diet_food_nutrients.csv",
             "values" : diet_food_nutrients            
        },
        {
            "id":"diet_nutrients.csv",
            "values" : diet_nutrients
        }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
    {
        "id":".*\.csv"
    }
    ]
}

Create a new job.

In [31]:
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print( job_uid )

90ab6fff-5770-465c-8984-7e8e4f81bc9a


Display job status until it is completed.

In [32]:
while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
completed


Display the KPI Total Calories value for this modified data. 

In [33]:
print( job_details['entity']['decision_optimization']['solve_state']['details']['KPI.Total Calories'] )

1499.9999999999998


In [34]:
print(client.deployments.get_job_details(job_uid)['entity']['decision_optimization']['status'])

{'state': 'completed', 'running_at': '2019-09-26T23:53:44.899Z', 'completed_at': '2019-09-26T23:53:45.742Z'}


### Delete the deployment

Use the following method to delete the deployment.

In [35]:
client.deployments.delete(deployment_uid)

'SUCCESS'

<a id='summary'></a>
### Summary and next steps

You successfully completed this notebook! 

You've learned how to:

- work with the Watson Machine Learning client
- prepare your model archive and upload your model on Watson Machine Learning
- create a deployment
- create and monitor a job with inline data for your deployed model
- display the solution

Check out our online documentation at <a href="https://dataplatform.cloud.ibm.com/docs" target="_blank" rel="noopener noreferrer">https://dataplatform.cloud.ibm.com/docs</a> for more samples, tutorials and documentation. 

<hr>
Copyright © 2019. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>